## Financial and technology articles taken from [webhose.io](https://webhose.io/datasets)

In [2]:
import pandas as pd
import json,urllib3
import glob
%matplotlib inline

## Take a look at one JSON file

In [23]:
with open('data/news_0000001.json','r', encoding='utf-8') as inFile:
    d=json.loads(inFile.read())

In [24]:
print(d.keys())

dict_keys(['organizations', 'uuid', 'thread', 'author', 'url', 'ord_in_thread', 'title', 'locations', 'entities', 'highlightText', 'language', 'persons', 'text', 'external_links', 'published', 'crawled', 'highlightTitle'])


In [25]:
print(d['text'])

Photo: Andrew Harnik, AP Image 1 of / 3 Caption
Close
Image 1 of 3 Pope Francis waves from a Fiat 500 as his motorcade departs from Andrews Air Force Base, Md., Tuesday, Sept. 22, 2015, where President and Mrs. Obama welcomed him.
Photo: Andrew Harnik, AP Pope Francis waves from a Fiat 500 as his motorcade departs from... Image 2 of 3 Pope Francis arrives at the Apostolic Nunciature, the Vatican's diplomatic mission in the heart of Washington, Tuesday, Sept. 22, 2015, in a Fiat 500. Pope Francis will visit the White House on Wednesday, becoming only the third pope to visit the White House.
Photo: Cliff Owen, AP Pope Francis arrives at the Apostolic Nunciature, the Vatican's... Image 3 of 3 Pope Francis waves from a Fiat 500L as his motorcade departs Andrews Air Force Base, Md., Tuesday, Sept. 22, 2015, after being greeted by President Barack Obama and first lady Michelle Obama.
Photo: Andrew Harnik, AP Pope Francis waves from a Fiat 500L as his motorcade departs... Who needs a limo? Po

## Define a function to open a file and get the text

In [26]:
def getText(f):
    with open(f,'r', encoding='utf-8') as inFile:
        d=json.loads(inFile.read())
    return d['text']

In [27]:
%time financeTexts=map(getText,glob.glob('../financial_news/data/news_*json'))

Wall time: 84 ms


In [29]:
print(financeTexts)
print(len(list(financeTexts)))

0


In [30]:
%time techTexts=map(getText,glob.glob('../tech_news/data/news*json'))

Wall time: 102 ms


In [31]:
(techTexts)

## Combine tech and financial news into one dataframe

In [33]:
df=pd.DataFrame(data={'text':list(financeTexts),'category':'finance'})

In [35]:
df=df.append(pd.DataFrame(data={'text':list(techTexts),'category':'tech'}))

In [36]:
df.head()

,text,category
0,"SAN MATEO, Calif. The Portland Trail Blazers a...",tech
1,The ANC has tightened the noose on members who...,tech
2,Nokia and the French Government announce agree...,tech
3,Submitted by admin on 9/21/2015 @ 4:20 PM Re...,tech
4,DNA analysis identifies long-lost remains of e...,tech


In [37]:
df.shape

(22292, 2)

In [38]:
df.head()

,text,category
0,"SAN MATEO, Calif. The Portland Trail Blazers a...",tech
1,The ANC has tightened the noose on members who...,tech
2,Nokia and the French Government announce agree...,tech
3,Submitted by admin on 9/21/2015 @ 4:20 PM Re...,tech
4,DNA analysis identifies long-lost remains of e...,tech


## Build up a pipeline

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn import preprocessing

## Binarise the category labels

In [40]:
lb = preprocessing.LabelBinarizer()

In [41]:
lb.fit(df['category'])
df['category_bin']=lb.transform(df['category'])

## Test Naive Bayes Classifier fr our baseline

In [42]:
steps=[('vectorise',CountVectorizer()),\
       ('transform',TfidfTransformer()),\
       ('clf',MultinomialNB())]
# Our pipeline has three steps

In [43]:
pipe=Pipeline(steps)

In [44]:
X_train, X_test, y_train, y_test=\
train_test_split(df['text'],df['category_bin'],test_size=0.25)

In [45]:
pipe.fit(X_train,y_train)

c:\program files\python37\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Pipeline(memory=None,
     steps=[('vectorise', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [46]:
pred=pipe.predict(X_test)

In [47]:
print 'Accuracy = %.3f' % f1_score(y_test,pred)

SyntaxError: invalid syntax (<ipython-input-47-1dd1aa4b4594>, line 1)

## Write out model

In [ ]:
import pickle
with open('model.out','w') as outFile:
    pickle.dump(pipe,outFile)

In [ ]:
lb

## Video 4.3

## Grid Search

In [48]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [49]:
pipe.named_steps.keys()

dict_keys(['vectorise', 'transform', 'clf'])

In [50]:
param_grid = dict(vectorise__stop_words=[None,'english'],\
                  vectorise__binary=[True,False],\
                  #vectorise__min_df=[1,5,10],\
                  #clf__class_weight=[None,'balanced'],\
                  #transform__norm=['l1','l2']
                 )

In [51]:
grid_search = GridSearchCV(pipe, param_grid=param_grid,\
                           scoring=make_scorer(f1_score),n_jobs=2)
# With n_jobs=1, takes 10.33
# With n_jobs=-1 takes YYYs

In [ ]:
%time res=grid_search.fit(df['text'],df['category_bin'])

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
res.best_params_

In [ ]:
print 'Best score = %.3f' % res.best_score_

## Compare classifiers

In [ ]:
CountVectorizer()

In [ ]:
for clf in [SGDClassifier(),LogisticRegression(),RandomForestClassifier()]:
    print clf.__class__
    steps=[('vectorise',CountVectorizer()),('transform',TfidfTransformer()),\
           ('clf',clf)]
    pipe=Pipeline(steps)
    pipe.set_params(vectorise__decode_error='ignore')
    
    grid_search = GridSearchCV(pipe, param_grid=param_grid,n_jobs=-1,\
                           scoring=make_scorer(f1_score))

    res=grid_search.fit(df['text'],df['category_bin'])
    
    print 'Best score = %.3f' % res.best_score_
    print res.best_params_
    print ''